In [39]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

train_topics = [u'freedom_of_speech', 
                u'human_rights', u'law',  
                u'world_international', 
                u'economy', u'culture', 
               ]

dev_topics = [  
    u'health_and_medicine', u'society',  u'science_and_technology',u'gender', u'education', 
]

test_topics = [
    u'politics',
    u'digital_freedom', 
    u'sports_and_entertainments', 
    u'religion', 
    u'environment', 
    u'philosophy', 
    u'ethics'
]

split_id = {}

total_train_size = 0; 
total_dev_size = 0; 
total_test_size = 0; 
for c in claims:
    cId = c['cId']
    topics = c['topics']
    claim_text = c['text']
#     for topic_text in topics:
    train_int = len(intersection(topics, train_topics))
    dev_int = len(intersection(topics, dev_topics))
#     assert train_int == 0 or dev_int == 0 
    if train_int > 0: 
        total_train_size += 1
        split_id[cId] = "train"
    elif dev_int > 0: 
        total_dev_size += 1
        split_id[cId] = "dev"
    else: 
        total_test_size += 1
        split_id[cId] = "test"
#         if topic_text in train_topics:
#             total_train_size += 1
#             break  

print(total_train_size)
print(total_dev_size)
print(total_test_size)
print(len(claims))
print(total_train_size * 1.0 / len(claims))
print(total_dev_size * 1.0 / len(claims))
print(total_test_size * 1.0 / len(claims))
print((total_train_size + total_test_size + total_dev_size)* 1.0 / len(claims))


import json
with open('/Users/daniel/ideaProjects/perspective/data/lucene_cach/dataset_split_v0.2.json', 'w') as outfile:
    json.dump(split_id, outfile)

541
139
227
907
0.596471885336
0.153252480706
0.250275633958
1.0


In [ ]:
# creating clusters of claims that do not share any perspectives

In [21]:
# OLD: create stance training data

# import json 

# with open('../data/dataset/perspectrum_with_answers_v0.1.json') as data:
#     all_annotations = json.load(data)

# with open('../data/dataset/perspective_pool_v0.1.json') as data:
#     perspectives = json.load(data)

# p_map = {}
# for p in perspectives: 
#     p_map[p['pId']] = p['text']


# # create pairs of claims and perspectices and save them in a csv file 

# import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')


# def save_in_file(claims, file_name): 
#     with open('../data/dataset/' + file_name, mode='w') as employee_file:
#         write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         for c in claims: 
#             claim_text = c["text"]
#             for p in c["perspectives"]:                
#                 for pid in p["pids"]: 
#                     if "SUPPORT" in p['stance_label_3']: 
#                         p_text = p_map[pid]
#                         print(pid)
#                         print(p_text)
#                         write.writerow([str(1), str(0), str(0), claim_text, p_text])  
#                     if "UNDERMINE" in p['stance_label_3']: 
#                         p_text = p_map[pid]
#                         write.writerow([str(0), str(0), str(0), claim_text, p_text])  
# split_idx = int(0.7 * len(all_annotations))  
        
# save_in_file(all_annotations[0:split_idx], 'perspective_stances/train.tsv')
# save_in_file(all_annotations[1 + split_idx:], 'perspective_stances/dev.tsv')

In [19]:
# Create stance training data

import json 

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

p_map = {}
for p in perspectives: 
    p_map[p['pId']] = p['text']
    
# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

def save_in_file(claims, file_name): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            claim_text = c["text"]
            for p in c["perspectives"]:                
                for pid in p["pids"]: 
                    if "SUPPORT" in p['stance_label_3']: 
                        p_text = p_map[pid]
#                         print(pid)
#                         print(p_text)
                        write.writerow([str(1), str(0), str(0), claim_text, p_text])  
                    if "UNDERMINE" in p['stance_label_3']: 
                        p_text = p_map[pid]
                        write.writerow([str(0), str(0), str(0), claim_text, p_text])  

train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

save_in_file(train_claims, 'perspective_stances/train.tsv')
save_in_file(test_claims, 'perspective_stances/test.tsv')
save_in_file(dev_claims, 'perspective_stances/dev.tsv')

In [100]:
# Create relevance training data

import json 
import random
import query_elasticsearch as es

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

all_pid_list = []
    
c_map ={}
for c in all_annotations:
    c_map[c["cId"]] = c["text"]
    
p_map = {}
for p in perspectives: 
    pid = p['pId']
    all_pid_list.append(pid)
    p_map[pid] = p['text']

related_p_map = {}
for c in all_annotations:
    cands = []
    cid = c["cId"]
    for cluster in c["perspectives"]:
        cands += cluster["pids"]
    
    related_p_map[cid] = set(cands)
    


# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

def save_in_file(claims, file_name, test_or_dev=False): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            count = 0
            cid = c["cId"]
            claim_text = c["text"]
            lucene_results = es.get_perspective_from_pool(claim_text, 50)
            
            rp = related_p_map[cid]
            
            if test_or_dev:
                pid_set = [_pid for _text, _pid, _score in lucene_results]
                for pid in pid_set:
                    p_text = p_map[pid]
                    if pid in rp:
                        write.writerow([str(1), str(cid), str(pid), claim_text, p_text])
                    else:
                        write.writerow([str(0), str(cid), str(pid), claim_text , p_text])
            else:            
                for pid in rp:
                    if pid in related_p_map[cid]:
                        p_text = p_map[pid]
                        write.writerow([str(1), str(cid), str(pid), claim_text, p_text])
                        count += 1

                for _text, pid, _score in lucene_results:
                    if pid not in related_p_map[cid]:
                        p_text = p_map[pid]
                        write.writerow([str(0), str(cid), str(pid), claim_text , p_text])
                        count -= 1
                        if count <= 0:
                            break
                    

# train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

# save_in_file(train_claims, 'perspective_relevance/train.tsv')
save_in_file(test_claims, 'perspective_relevance/test.tsv', test_or_dev=True)
save_in_file(dev_claims, 'perspective_relevance/dev.tsv', test_or_dev=True)

In [11]:
####
# Evaluate relevance
####

import csv
import pandas as pd

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

claim_map = {}
for c in all_annotations:
    claim_map[c["cId"]] = c
    
raw_score = '../data/dataset/perspective_relevance/eval_logits.out'
test_data = '../data/dataset/perspective_relevance/test.tsv'

test_ids = []
reader = csv.reader(open(test_data), delimiter="\t", quotechar='"')
for line in reader:
    test_ids.append([int(line[1]), int(line[2])])
        

rdf = pd.read_csv(raw_score, header=None, names=["no_score", "yes_score"])
rdf["cid"] = rdf['pid1'] = rdf['pid2'] = rdf['pred'] = 0 
rdf.info()

for idx, row in rdf.iterrows():
    rdf.at[idx, 'cid'] = test_ids[idx][0]
    rdf.at[idx, 'pid'] = test_ids[idx][1]
    rdf.at[idx, 'pred'] = 1 if row.no_score < row.yes_score else 0

rdf.to_csv('../data/dataset/perspective_relevance/test_raw_score_w_id.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2431 entries, 0 to 2430
Data columns (total 6 columns):
no_score     2431 non-null float64
yes_score    2431 non-null float64
cid          2431 non-null int64
pid1         2431 non-null int64
pid2         2431 non-null int64
pred         2431 non-null int64
dtypes: float64(2), int64(4)
memory usage: 114.0 KB


In [12]:
####
# Evaluate relevance
####
related_p_map = {}
for c in all_annotations:
    cands = []
    cid = c["cId"]
    for cluster in c["perspectives"]:
        cands += cluster["pids"]
    
    related_p_map[cid] = set(cands)
    
unique_cids = rdf.cid.unique()

num_lucene_cands = [x for x in range(1, 20)]

best_param = -1
best_p = -1
best_r = -1
best_f1 = -1

for num_c in num_lucene_cands:
    tot_p = tot_r = tot_count = 0
    for cid in unique_cids:
        claim = claim_map[cid]
        covered = [False for _c in claim["perspectives"]]
        
        cdf = rdf[rdf.cid == cid].head(num_c)

        for idx, row in cdf.iterrows():
            
            pid = row.pid
            
            for idx, clus in enumerate(claim["perspectives"]):
                if pid in clus["pids"]:
                    covered[idx] = True

        tot_pred = len(cdf.index)
        tot_gold = len(covered)
        hit = [h for h in covered if h]
        
        if tot_pred == 0:
            tot_p += 1
        else:
            tot_p += len(hit) / tot_pred

        if tot_gold == 0:
            tot_r += 1
        else:
            tot_r += len(hit) / tot_gold


    mean_p = tot_p / len(unique_cids)
    mean_r = tot_r / len(unique_cids)
    mean_f1 = 2 * mean_p * mean_r / (mean_p + mean_r)
    
    print(num_c, mean_p, mean_r, mean_f1)
    if mean_f1 > best_f1:
        best_f1 = mean_f1
        best_p = mean_p
        best_r = mean_r
        best_param = num_c

print("Best precision: {}".format(best_p))
print("Best Recall: {}".format(best_r))
print("Best F1: {}".format(best_f1))
print("Best # candidates: {}".format(best_param))

1 0.8281938325991189 0.23854238728422147 0.37039959885091006
2 0.6299559471365639 0.33080962721140844 0.4338113221287824
3 0.5411160058737156 0.38178773937049215 0.4476988151457768
4 0.47687224669603523 0.42069362442754804 0.4470248263792118
5 0.4396475770925107 0.44604423189787545 0.4428228054937776
6 0.4126284875183554 0.4656845716456879 0.43755405536536485
7 0.3937486889028739 0.47931865626830233 0.4323402851250338
8 0.375734214390602 0.4905334969460526 0.42552715677907105
9 0.36447626040137054 0.5013863813341453 0.42210721302990495
10 0.3523861967694568 0.5074388657021872 0.41593216988530846
11 0.34317514350553996 0.5133140962333826 0.4113458300818915
12 0.3354992657856094 0.519189485685442 0.40760496953135283
13 0.32736642945894057 0.5216121604513063 0.4022676485807741
14 0.32039542689322426 0.5247126981391381 0.3978557155870366
15 0.3143958464443046 0.5265136828502033 0.39370160336528587
16 0.3083202223620726 0.5270031591106046 0.3890367127423959
17 0.3037205851503597 0.528458706

In [98]:
# Create Evidence training data

import json 
import random
import query_elasticsearch as es

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/evidence_pool_v0.2.json', encoding='utf-8') as data:
    evidences = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

all_pid_list = []
    
c_map ={}
for c in all_annotations:
    c_map[c["cId"]] = c["text"]
    
p_map = {}
for p in perspectives: 
    pid = p['pId']
    all_pid_list.append(pid)
    p_map[pid] = p['text']
    
e_map = {}
for e in evidences: 
    e_map[e["eId"]] = e['text']

related_p_map = {}
for c in all_annotations:
    cands = []
    cid = c["cId"]
    for cluster in c["perspectives"]:
        cands += cluster["pids"]
    
    related_p_map[cid] = set(cands)
    


# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')


def save_in_file(claims, file_name, test_or_dev=False): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            count = 0
            cid = c["cId"]
            claim_text = c["text"]
            
            for cluster in c["perspectives"]:
                for pid in cluster["pids"]:
                    p_text = p_map[pid]
                    c_p_text = claim_text + '. ' + p_text
                    lucene_results = es.get_evidence_from_pool(c_p_text, 20)
                    
                    if test_or_dev:
                        for e_text, _eid, _score in lucene_results:
                            if _eid in cluster['evidence']:
                                write.writerow([str(1), str(cid) + '-' + str(pid), str(_eid), c_p_text, e_text])
                            else:
                                write.writerow([str(0), str(cid) + '-' + str(pid), str(_eid), c_p_text, e_text])
                    else:
                        for eid in cluster["evidence"]:
                            write.writerow([str(1), str(cid) + '-' + str(pid), str(eid), c_p_text, e_map[eid]])
                            count += 1
                        
                        for e_text, _eid, _score in lucene_results:
                            if _eid not in cluster['evidence']:
                                if count <= 0 and not test_or_dev:
                                    break
                                write.writerow([str(0), str(cid) + '-' + str(pid), str(_eid), c_p_text, e_text])
                                count -= 1
                            
                    

train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
# test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
# dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

save_in_file(train_claims, 'perspective_evidence/train.tsv', test_or_dev=True)
# save_in_file(test_claims, 'perspective_evidence/test.tsv', test_or_dev=True)
# save_in_file(dev_claims, 'perspective_evidence/dev.tsv', test_or_dev=True)

GET http://bronte.cs.illinois.edu:8080/evidence_pool_v0.2/text/_search?size=20 [status:N/A request:0.027s]
Traceback (most recent call last):
  File "/home/squirrel/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/squirrel/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/squirrel/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "/home/squirrel/anaconda3/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/home/squirrel/anaconda3/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/home/squirrel/anaconda3/lib/python3.6/http/client.py", line 266, in _read_status
    raise RemoteDisconnected("Remote end closed co

In [113]:
# Evaluate BERT results on evidence discovery
import csv
import pandas as pd

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

claim_map = {}
for c in all_annotations:
    claim_map[c["cId"]] = c
    
raw_score = '../data/dataset/perspective_evidence/test_raw_score.csv'
test_data = '../data/dataset/perspective_evidence/test.tsv'

test_ids = []
reader = csv.reader(open(test_data), delimiter="\t", quotechar='"')
for line in reader:
    pid_cid = [int(x) for x in line[1].split('-')]
    pid_cid.append(int(line[2]))
    pid_cid.append(line[3])
    pid_cid.append(line[4])
    test_ids.append(pid_cid)
        
test_ids.pop(0) # somehow the bert script ignores the first instance during reading


rdf = pd.read_csv(raw_score)
rdf["cid"] = rdf['pid'] = rdf['eid'] = rdf['pred'] = 0 
rdf["c_p_text"] = rdf["e_text"] = ""
rdf.info()

for idx, row in rdf.iterrows():
    rdf.at[idx, 'cid'] = test_ids[idx][0]
    rdf.at[idx, 'pid'] = test_ids[idx][1]
    rdf.at[idx, 'eid'] = test_ids[idx][2]
    rdf.at[idx, "c_p_text"] = test_ids[idx][3]
    rdf.at[idx, "e_text"] = test_ids[idx][4]
    rdf.at[idx, 'pred'] = 1 if row.undermine_score < row.support_score else 0
    
rdf.to_csv('../data/dataset/perspective_evidence/test_raw_score_w_id.csv', index=False)
    


FileNotFoundError: [Errno 2] No such file or directory: '../data/dataset/perspective_evidence/test.tsv'

In [114]:
# Evaluate BERT results on evidence discovery, cont...

unique_cids = rdf.cid.unique()

num_lucene_cands = [x for x in range(1, 20)]

best_param = -1
best_p = -1
best_r = -1
best_f1 = -1

for num_c in num_lucene_cands:
    tot_p = tot_r = tot_count = 0
    for cid in unique_cids:
        cdf = rdf[rdf.cid == cid]
        
        unique_pids = cdf.pid.unique()
        for pid in unique_pids:
            
            tp = tot_pred = tot_gold = 0
            pdf = cdf[cdf.pid == pid].head(num_c)
            
            for idx, row in pdf.iterrows():

                if row.pred == 1:
                    tot_pred += 1
                    if row.gold == 1:
                        tp += 1
                pid = row.pid

                for cluster in claim_map[cid]["perspectives"]:
                    if pid in cluster["pids"]:
                        tot_gold = len(cluster["evidence"])
                        break

            if tot_pred == 0:
                tot_p += 1
            else:
                tot_p += tp / tot_pred

            if tot_gold == 0:
                tot_r += 1
            else:
                tot_r += tp / tot_gold

            tot_count += 1


    mean_p = tot_p / tot_count
    mean_r = tot_r / tot_count
    mean_f1 = 2 * mean_p * mean_r / (mean_p + mean_r)
    
    print(num_c, mean_p, mean_r, mean_f1)
    if mean_f1 > best_f1:
        best_f1 = mean_f1
        best_p = mean_p
        best_r = mean_r
        best_param = num_c

print("Best precision: {}".format(best_p))
print("Best Recall: {}".format(best_r))
print("Best F1: {}".format(best_f1))
print("Best # candidates: {}".format(best_param))

2 0.7678524141583547 0.9460445549144916 0.8476852558825935
3 0.7678524141583547 0.9460445549144916 0.8476852558825935
4 0.7678524141583547 0.9460445549144916 0.8476852558825935
5 0.7678524141583547 0.9460445549144916 0.8476852558825935
6 0.7678524141583547 0.9460445549144916 0.8476852558825935
7 0.7678524141583547 0.9460445549144916 0.8476852558825935
8 0.7678524141583547 0.9460445549144916 0.8476852558825935
9 0.7678524141583547 0.9460445549144916 0.8476852558825935
10 0.7678524141583547 0.9460445549144916 0.8476852558825935
11 0.7678524141583547 0.9460445549144916 0.8476852558825935
12 0.7678524141583547 0.9460445549144916 0.8476852558825935
13 0.7678524141583547 0.9460445549144916 0.8476852558825935
14 0.7678524141583547 0.9460445549144916 0.8476852558825935
15 0.7678524141583547 0.9460445549144916 0.8476852558825935


KeyboardInterrupt: 

In [2]:
##### 
# Create equivalence training data
#####

import json
from itertools import combinations

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    

all_pid_list = []
    
c_map ={}
for c in all_annotations:
    c_map[c["cId"]] = c["text"]
    
p_map = {}
for p in perspectives: 
    pid = p['pId']
    all_pid_list.append(pid)
    p_map[pid] = p['text']

related_p_map = {}
for c in all_annotations:
    cands = []
    cid = c["cId"]
    for cluster in c["perspectives"]:
        cands += cluster["pids"]
    
    related_p_map[cid] = set(cands)
    


# create pairs of claims and perspectices and save them in a csv file 

import csv

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

def save_in_file(claims, file_name, test_or_dev=False): 
    with open('../data/dataset/' + file_name, mode='w') as employee_file:
        write = csv.writer(employee_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for c in claims: 
            count = 0
            cid = c["cId"]
            claim_text = c["text"]
            
            pairs = list(combinations(related_p_map[cid], 2))
            
            if test_or_dev:
                for p1, p2 in pairs:
                    p1_text = p_map[p1]
                    p2_text = p_map[p2]
                    same_cluster = False
                    for cluster in c["perspectives"]:
                        if (p1 in cluster["pids"]) and (p2 in cluster["pids"]):
                            same_cluster = True
                            break
                    
                    if same_cluster:
                        write.writerow([str(1), str(cid) + '-' + str(p1), str(p2), claim_text + '. ' + p1_text, p2_text])
                    else:
                        write.writerow([str(0), str(cid) + '-' + str(p1), str(p2), claim_text + '. ' + p1_text, p2_text])
            else:
                counter = 0
                for p1, p2 in pairs:
                    p1_text = p_map[p1]
                    p2_text = p_map[p2]
                    same_cluster = False
                    for cluster in c["perspectives"]:
                        if (p1 in cluster["pids"]) and (p2 in cluster["pids"]):
                            same_cluster = True
                            break
                    
                    if same_cluster:
                        write.writerow([str(1), str(cid) + '-' + str(p1), str(p2), claim_text + '. ' + p1_text, p2_text])
                        counter += 1
                    elif counter > 0:
                        write.writerow([str(0), str(cid) + '-' + str(p1), str(p2), claim_text + '. ' + p1_text, p2_text])
                        counter -= 1

            
#             if test_or_dev:
#                 pid_set = [_pid for _text, _pid, _score in lucene_results]
#             else:
#                 pid_set = related_p_map[cid]
                
#             for pid in pid_set:
#                 if pid in related_p_map[cid]:
#                     p_text = p_map[pid]
#                     write.writerow([str(1), str(cid), str(pid), claim_text, p_text])
#                     count += 1
            
#             for _text, pid, _score in lucene_results:
#                 if pid not in related_p_map[cid]:
#                     p_text = p_map[pid]
#                     write.writerow([str(0), str(cid), str(pid), claim_text , p_text])
#                     count -= 1
#                     if count <= 0:
#                         break
                    

train_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'train']
test_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'test']
dev_claims = [_c for _c in all_annotations if split[str(_c['cId'])] == 'dev']

save_in_file(train_claims, 'perspective_equivalence/train.tsv', test_or_dev=True)
save_in_file(test_claims, 'perspective_equivalence/test.tsv', test_or_dev=True)
save_in_file(dev_claims, 'perspective_equivalence/dev.tsv', test_or_dev=True)

In [18]:
####
# Evaluate equivalence
####

import csv
import pandas as pd

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

claim_map = {}
for c in all_annotations:
    claim_map[c["cId"]] = c
    
raw_score = '../result/lr3e-05_bs32_equivalence_eval_logits.out'
test_data = '../data/dataset/perspective_equivalence/test.tsv'

test_ids = []
reader = csv.reader(open(test_data), delimiter="\t", quotechar='"')
for line in reader:
    pid_cid = [int(x) for x in line[1].split('-')]
    pid_cid.append(int(line[2]))
    pid_cid.append(int(line[0]))
    test_ids.append(pid_cid)
        
# test_ids.pop(0) # somehow the bert script ignores the first instance during reading


rdf = pd.read_csv(raw_score, header=None, names=["undermine_score", "support_score"])
rdf["cid"] = rdf['pid1'] = rdf['pid2'] = rdf['pred'] = rdf['gold'] = 0 
rdf.info()

for idx, row in rdf.iterrows():
    rdf.at[idx, 'cid'] = test_ids[idx][0]
    rdf.at[idx, 'pid1'] = test_ids[idx][1]
    rdf.at[idx, 'pid2'] = test_ids[idx][2]
    rdf.at[idx, 'gold'] = test_ids[idx][3]
    rdf.at[idx, 'pred'] = 1 if row.undermine_score < row.support_score else 0
    
rdf.to_csv('../data/dataset/perspective_equivalence/test_raw_score_w_id.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33571 entries, 0 to 33570
Data columns (total 7 columns):
undermine_score    33571 non-null float64
support_score      33571 non-null float64
cid                33571 non-null int64
pid1               33571 non-null int64
pid2               33571 non-null int64
pred               33571 non-null int64
gold               33571 non-null int64
dtypes: float64(2), int64(5)
memory usage: 1.8 MB


In [19]:
####
# Evaluate equivalence cont.
####

unique_cids = rdf.cid.unique()

tot_p = tot_r = 0
for cid in unique_cids:
    cdf = rdf[rdf.cid == cid]
    tp = tot_pred = tot_gold = 0
    for idx, row in cdf.iterrows():
        if row.gold == 1:
            tot_gold += 1

        if row.pred == 1:
            tot_pred += 1
            if row.gold == 1:
                tp += 1

    if tot_gold == 0:
        tot_p += 1
    else:
        tot_p += tp / tot_gold

    if tot_pred == 0:
        tot_r += 1
    else:
        tot_r += tp / tot_pred

mean_p = tot_p / len(unique_cids)
mean_r = tot_r / len(unique_cids)
mean_f1 = 2 * mean_p * mean_r / (mean_p + mean_r)
    

print("Precision: {}".format(mean_p))
print("Recall: {}".format(mean_r))
print("F1: {}".format(mean_f1))


Precision: 0.7626610615157229
Recall: 0.7431619325662322
F1: 0.7527852484608711


In [ ]:
"""
Evidence Discovery Task

Making training data for 
"""
import json

with open('../data/dataset/perspectrum_with_answers_v0.2.json', encoding='utf-8') as data:
    all_annotations = json.load(data)

with open('../data/dataset/perspective_pool_v0.2.json', encoding='utf-8') as data:
    perspectives = json.load(data)

with open('../data/dataset/dataset_split_v0.2.json', encoding='utf-8') as data:
    split = json.load(data)
    
with open('../data/dataset/evidence_pool_v0.2.json', encoding='utf-8') as data:
    evidences = json.load(data)
    
# First split the evidence into batches of 3 sentences. 


### New BERT baseline results

| Task        | P | R | F1 |
|-------------|---|---|----|
| Relevance   | ~54.1  | ~38.2  | ~44.3   |
| Stance      | 78.3 | 76.7 | 77.5 |
| Equivalence | 76.3 | 74.3 | 75.3 |
| Evidence    | ~60.2 | ~37.9 | ~46.5 |
